In [ ]:
import matplotlib.pyplot as plt
import os
import time
import matplotlib
import numpy as np
import pandas as pd

Optimisation in science
<div>
<img src="http://www.jensuhlig.de/Kemm30/KEMM30_007.png" width="600">
</div>

How do you get there? or better,how do you get there without producing a lot of nonsense
Counting parameter. e.g. 10 peaks, each position, width, intensity =30 parameter plus background. So fitting is about intelligence. Think, $\textbf{optimize}$ the smallest amount of parameter starting with a good guess

One parameter optimisation (middle = mu)
<div>
<img src="http://www.jensuhlig.de/Kemm30/error_way.png" width="1000">
</div>

<div>
<img src="http://www.jensuhlig.de/Kemm30/KEMM30_008.jpg" width="900">
</div>

Before we can go into optimisation we have to talk about 

# Functions:

Two ways to define functions: the classical way using def a more compact way is using a so-called lambda function. We will focus on the definition only

In [ ]:
def my_function():  #my_function is the name of the function. The brackets are mandatory 
    print('Hello World')

with the above command you only define a function, but do not execute it. this you do by writing the function name with round brackets

In [ ]:
my_function()

In [ ]:
def my_function2(s):  #this function has an input
    print(s)
my_function2(s='Hello world') #call the function with defined parameter
my_function2('Hello world') # or position parameters

important, what is in a function stays in a function. meaning this line will give an error

In [ ]:
def my_function2(s):  #this function has an input
    a_parameter=5
a_parameter

if you want to use something out of a function you have to return it. Usually you would assign the returned value to a variable, here "catch_the_output".

In [ ]:
def my_function3(s): 
    return s+1
catch_the_output=my_function3(2)

In [ ]:
def my_function4(s): 
    '''this fucntion returns multiple things'''
    return s,s+1
catch_the_output1,catch_the_output2=my_function4(10)
print(catch_the_output1,' and ', catch_the_output2)

### Task:

* Write a standard function that takes x, mu and sigma and returns a gaussian bell curve (with normalisation). Make sure that you use numpy operations only, so that you can give it a vector and receive a vector.<br> 
${\frac {1}{\sqrt {2\pi \sigma ^{2}}}}\operatorname {exp} \left(-{\frac {\left(x-\mu \right)^{2}}{2\sigma ^{2}}}\right)$
* plot this function with mu=0,sigma=1 from -5 to 5 in 0.01 steps 

* Write a function that returns the normalized vector 
* use the cumsum function from numpy to create the cummulative sum of the gaussian from above and normalize it with your lambda function
* Plot this in the same plot (against x)
* What does this resemble?

# Parameter optimisation: Fitting vs. optimisation


## Curve Fit
Starting with curve_fit. We assume that we have a flat function that has a clear gradient to the minimum (see top of this page) then we can use curve_fit to fast measure the parameter we need

In [ ]:
gauss = lambda x,mu=0,sigma=1,offset=0: 1/np.sqrt(2*np.pi*(sigma**2))*np.exp((-0.5/sigma**2)*(np.subtract(x,mu))**2)+offset
                                    
#create some data with some randomness and plot it
x=np.linspace(-5,5,200)
y=gauss(x,mu=0.5,sigma=0.5)+0.1*np.random.random(np.shape(x))
fig,ax=plt.subplots()
ax.plot(x,y,'o',ms=2,label='data')
ax.set_xlabel('measured value')
ax.set_ylabel('occurance')

In [ ]:
from scipy.optimize import curve_fit
#make a guess
p0=[1,1,0.1]

#optimise
popt,pcov = curve_fit(gauss, xdata=x, ydata=y,p0=p0)

#plot both
ax.plot(x, gauss(x, mu=p0[0],sigma=p0[1],offset=p0[2]), 'b-', label='guess')
ax.plot(x, gauss(x, mu=popt[0],sigma=popt[1],offset=popt[2]), 'r-', label='fit')
ax.legend()

#get errors from the covariance matrix (works here, but careful)
perr = np.sqrt(np.diag(pcov))
df=pd.DataFrame({'values':popt,'errors':perr},index=['$\mu$','$\sigma$','$x_0$'])
df=df[['values','errors']]
df

The key bit
```
from scipy.optimize import curve_fit
popt,pcov = curve_fit(gauss, xdata=x, ydata=y,p0=[1,0.5,0.1])
```
curve fit is a least square method that takes a function, the target data and a set of starting parameters, that are in order the parameter after the first.
it returns: 
popt = optimized parameter
and
pcov = covariance matrix.
p_sigma = np.sqrt(np.diag(pcov))


<div>
<img src="http://www.jensuhlig.de/Kemm30/KEMM30_009.jpg" width="400">
</div>

### Tasks:

Read the file, Fit the file, plot both, data and fits:
What is the center position of the peak
http://www.jensuhlig.de/Kemm30/fit_0.csv

## Scipy minimize

Second way of optimisation uses in addition to the "cost function" an "error function". The task of the second function is to create a "price" for this parameter. The Minimize function is then minimizing this price.

In [ ]:
def gauss(x,p): #the function that is your model
    [mu,sigma,offset]=p
    pre_factor=1/np.sqrt(2*np.pi*(sigma**2))
    exponent=(-0.5/sigma**2)*(np.subtract(x,mu))**2
    return pre_factor*np.exp(exponent)+offset
x=np.linspace(-5,5,200)
y=gauss(x,[0.5,0.5,0.2])
y=y+(y**0.5)*0.5*np.random.random(np.shape(x))
fig,ax=plt.subplots(figsize=(4,4))
ax.plot(x,y,'o',ms=5,label='data')
ax.set_xlabel('measured value')
ax.set_ylabel('occurance')

The new thing is that you need a second function that produces you a single "error" value

In [ ]:
def min_gauss(p):# this would be the root mean square (we skip the root as the minimum is the minimum)
    return ((y-gauss(x,p))**2).sum()
def min_gauss_lin(p): #this is a different cost function that uses a more linear approach. It does not "punish" strong deviations as much. As such it has the tendency to be more outlier stable but preforms bad for peaks.
    return np.abs(y-gauss(x,p)).sum()

In [ ]:
from scipy.optimize import minimize #import the minimization function
x0=[1.,0.5,0.1]
out = minimize(min_gauss,x0=x0,method='Nelder-Mead') #nelder-Mead is a standard multi-parameter optimiser. check out other choices.
out2 = minimize(min_gauss_lin,x0=x0,method='Nelder-Mead')

In [ ]:
fig,ax=plt.subplots(figsize=(8,8))
ax.plot(x,y,'o',ms=5,label='data')
ax.set_xlabel('measured value')
ax.set_ylabel('occurance')
ax.plot(x, gauss(x, p=x0), color='blue',lw=5,alpha=0.5,label='start')
ax.plot(x, gauss(x, p=out['x']), color='red',linestyle='dashed', lw=5,label='fit_sqrt')
ax.plot(x, gauss(x, p=out2['x']), color='green',lw=5,zorder=0, label='fit_lin')
print(out)
plt.legend()

### Tasks:

Read the files, Fit files and plot both, data and fits:
* http://www.jensuhlig.de/Kemm30/fit_0.csv
* http://www.jensuhlig.de/Kemm30/fit_1.csv  here: try first a separate fit, in which you fit the linear range and then separately the peak.
* http://www.jensuhlig.de/Kemm30/fit_3.csv (two peaks and background)

### Fit the peaks

<div>
<img src="http://www.jensuhlig.de/Kemm30/2D_measured_indicated.png" width="200">
</div>

# Advanced

While scipy minimize is a very useful tool, it is still a bit difficult to handle parameters. A very nice tool that was developed for this lmfit.

In [ ]:
import lmfit

def gauss_with_names(x,par): #the function that is your model
    pre_factor=1/np.sqrt(2*np.pi*(par['sigma']**2))
    exponent=(-0.5/par['sigma']**2)*(np.subtract(x,par['mu']))**2
    return pre_factor*np.exp(exponent)+par['offset']
def min_gauss(par,x,y):# this would be the root mean square (we skip the root as the minimum is the minimum)
    return ((y-gauss_with_names(x,par))**2).sum()

x=np.linspace(-5,5,200)
y=gauss(x,[0.5,0.5,0.2])
y=y+(y**0.5)*0.5*np.random.random(np.shape(x))


#first create a parameter object
par=lmfit.Parameters()                                       # create empty parameter object

par.add('mu',value=0,vary=True)                                # Add a parameter
par.add('sigma',value=0.5,vary=True)
par.add('offset',value=0.1,vary=True)

mini = lmfit.Minimizer(min_gauss,par,fcn_kws={'x':x,'y':y})
results = mini.minimize('nelder')
results